# SQL Data Analysis from Data Warehouse

In [44]:
import os
import boto3
import duckdb

## Download the Data from the s3 source

### **Note**: You must acctive the dag analytics_warehouse_sql_dag before you download it from s3

In [2]:
if not os.path.exists('business_analytics.db'):
    s3 = boto3.client(
        service_name="s3",
        endpoint_url="http://localhost:9000",
        aws_access_key_id="minioLocalAccessKey",
        aws_secret_access_key="minioLocalSecretKey123",
        region_name="us-east-1",
    )

    s3.download_file(
        Bucket='business-analytics-sql',
        Key='warehouse/business_analytics.db',
        Filename='business_analytics.db'
    )

con = duckdb.connect(database='business_analytics.db')

## Exploratory Data Analysis

### 1. Database Exploration

This is the first step in any data analysis process. It involves getting familiar with the structure and contents of your database. You check which tables exist, what types of data they hold, how they're connected, and whether there are missing or duplicate values.
The goal is to understand the "shape" of your data before diving into deeper analysis.

In [3]:
schema_list = con.execute("SELECT table_schema, table_name FROM information_schema.tables").df()
schema_list

,table_schema,table_name
0,bronze,cat_erp
1,bronze,cust_erp
2,bronze,cust_info
3,bronze,loc_erp
4,bronze,prd_info
5,bronze,sales_details_info
6,silver,crm_cust_info
7,silver,crm_prd_info
8,silver,crm_sales_details
9,silver,erp_cust


In [4]:
column_list = con.execute("SELECT * FROM information_schema.columns WHERE table_name = 'dim_customers'").df()
column_list

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,business_analytics,gold,dim_customers,customer_key,1,None,YES,BIGINT,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
1,business_analytics,gold,dim_customers,customer_id,2,None,YES,INTEGER,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
2,business_analytics,gold,dim_customers,customer_number,3,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
3,business_analytics,gold,dim_customers,first_name,4,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
4,business_analytics,gold,dim_customers,last_name,5,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
5,business_analytics,gold,dim_customers,country,6,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
6,business_analytics,gold,dim_customers,marital_status,7,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
7,business_analytics,gold,dim_customers,gender,8,None,YES,VARCHAR,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
8,business_analytics,gold,dim_customers,birthdate,9,None,YES,DATE,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None
9,business_analytics,gold,dim_customers,create_date,10,None,YES,DATE,<NA>,<NA>,...,None,None,None,None,None,<NA>,None,None,<NA>,None


In [5]:
gold_dim_customers = con.execute("SELECT * FROM gold.dim_customers").df()
gold_dim_customers

,customer_key,customer_id,customer_number,first_name,last_name,country,marital_status,gender,birthdate,create_date,dwh_create_date
0,1,11000,AW00011000,Jon,Yang,Australia,Married,Male,1971-10-06,2025-10-06,2025-05-25 15:28:04.496
1,2,11001,AW00011001,Eugene,Huang,Australia,Single,Male,1976-05-10,2025-10-06,2025-05-25 15:28:04.496
2,3,11002,AW00011002,Ruben,Torres,Australia,Married,Male,1971-02-09,2025-10-06,2025-05-25 15:28:04.496
3,4,11003,AW00011003,Christy,Zhu,Australia,Single,Female,1973-08-14,2025-10-06,2025-05-25 15:28:04.496
4,5,11004,AW00011004,Elizabeth,Johnson,Australia,Single,Female,1979-08-05,2025-10-06,2025-05-25 15:28:04.496
...,...,...,...,...,...,...,...,...,...,...,...
18479,18480,29479,AW00029479,Tommy,Tang,France,Married,n/a,1969-06-30,2026-01-25,2025-05-25 15:28:04.496
18480,18481,29480,AW00029480,Nina,Raji,United Kingdom,Single,n/a,1977-05-06,2026-01-25,2025-05-25 15:28:04.496
18481,18482,29481,AW00029481,Ivan,Suri,Germany,Single,n/a,1965-07-04,2026-01-25,2025-05-25 15:28:04.496
18482,18483,29482,AW00029482,Clayton,Zhang,France,Married,n/a,1964-09-01,2026-01-25,2025-05-25 15:28:04.496


In [6]:
gold_dim_products = con.execute("SELECT * FROM gold.dim_products").df()
gold_dim_products

,product_key,product_id,product_number,product_name,category_id,category,subcategory,maintenance,cost,product_line,start_date,end_date,dwh_create_date
0,1,210,FR-R92B-58,HL Road Frame - Black- 58,CO_RF,Components,Road Frames,true,0,Road,2003-07-01,NaT,2025-05-25 15:28:04.543
1,2,211,FR-R92R-58,HL Road Frame - Red- 58,CO_RF,Components,Road Frames,true,0,Road,2003-07-01,NaT,2025-05-25 15:28:04.543
2,3,348,BK-M82B-38,Mountain-100 Black- 38,BI_MB,Bikes,Mountain Bikes,true,1898,Mountain,2011-07-01,NaT,2025-05-25 15:28:04.543
3,4,349,BK-M82B-42,Mountain-100 Black- 42,BI_MB,Bikes,Mountain Bikes,true,1898,Mountain,2011-07-01,NaT,2025-05-25 15:28:04.543
4,5,350,BK-M82B-44,Mountain-100 Black- 44,BI_MB,Bikes,Mountain Bikes,true,1898,Mountain,2011-07-01,NaT,2025-05-25 15:28:04.543
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,291,530,TT-T092,Touring Tire Tube,AC_TT,Accessories,Tires and Tubes,true,2,Touring,2013-07-01,NaT,2025-05-25 15:28:04.543
291,292,473,VE-C304-L,Classic Vest- L,CL_VE,Clothing,Vests,false,24,Other Sales,2013-07-01,NaT,2025-05-25 15:28:04.543
292,293,472,VE-C304-M,Classic Vest- M,CL_VE,Clothing,Vests,false,24,Other Sales,2013-07-01,NaT,2025-05-25 15:28:04.543
293,294,471,VE-C304-S,Classic Vest- S,CL_VE,Clothing,Vests,false,24,Other Sales,2013-07-01,NaT,2025-05-25 15:28:04.543


In [7]:
gold_fact_sales = con.execute("SELECT * FROM gold.fact_sales").df()
gold_fact_sales

,order_number,product_key,customer_key,order_date,shipping_date,due_date,sales_amount,quantity,price,dwh_create_date
0,SO43697,20,10769,2010-12-29,2011-01-05,2011-01-10,3578,1,3578,2025-05-25 15:28:04.553
1,SO43698,9,17390,2010-12-29,2011-01-05,2011-01-10,3400,1,3400,2025-05-25 15:28:04.553
2,SO43699,9,14864,2010-12-29,2011-01-05,2011-01-10,3400,1,3400,2025-05-25 15:28:04.553
3,SO43700,41,3502,2010-12-29,2011-01-05,2011-01-10,699,1,699,2025-05-25 15:28:04.553
4,SO43701,9,4,2010-12-29,2011-01-05,2011-01-10,3400,1,3400,2025-05-25 15:28:04.553
...,...,...,...,...,...,...,...,...,...,...
60374,SO75122,174,4869,2014-01-28,2014-02-04,2014-02-09,22,1,22,2025-05-25 15:28:04.553
60375,SO75122,166,4869,2014-01-28,2014-02-04,2014-02-09,9,1,9,2025-05-25 15:28:04.553
60376,SO75123,174,7760,2014-01-28,2014-02-04,2014-02-09,22,1,22,2025-05-25 15:28:04.553
60377,SO75123,281,7760,2014-01-28,2014-02-04,2014-02-09,159,1,159,2025-05-25 15:28:04.553


### 2. Dimensions Exploration

Dimensions are descriptive fields like customer name, product category, or region. Exploring dimensions means looking at the variety and frequency of these fields to uncover patterns.

For example, you might check:

- How many product categories exist

- Which regions have the most sales

- How customers are distributed by gender or age group



In [8]:
country_list = con.execute('SELECT DISTINCT country FROM gold.dim_customers').df()
country_list

,country
0,United Kingdom
1,Germany
2,Australia
3,United States
4,France
5,Canada
6,n/a


In [9]:
categories_list = con.execute('SELECT DISTINCT category, subcategory, product_name FROM gold.dim_products ORDER BY 1,2,3').df()
categories_list

,category,subcategory,product_name
0,Accessories,Bike Racks,Hitch Rack - 4-Bike
1,Accessories,Bike Stands,All-Purpose Bike Stand
2,Accessories,Bottles and Cages,Mountain Bottle Cage
3,Accessories,Bottles and Cages,Road Bottle Cage
4,Accessories,Bottles and Cages,Water Bottle - 30 oz.
...,...,...,...
290,n/a,n/a,LL Mountain Pedal
291,n/a,n/a,LL Road Pedal
292,n/a,n/a,ML Mountain Pedal
293,n/a,n/a,ML Road Pedal


### 3. Date Exploration

This focuses on understanding how data behaves over time. It includes identifying the date range of the dataset, spotting gaps in time, and understanding seasonality or trends.

Key questions might be:

- What is the time span of the dataset?

- Are there missing days or months?

- Is there a weekly/monthly/seasonal pattern?

In [10]:
date = con.execute("SELECT MIN(order_date) as first_order_date, MAX(order_date) as last_order_date, DATEDIFF('year', MIN(order_date), MAX(order_date)) as order_range_years FROM gold.fact_sales").df()
date

,first_order_date,last_order_date,order_range_years
0,2010-12-29,2014-01-28,4


In [11]:
date_customer = con.execute("SELECT MIN(birthdate) as oldest_birthdate, DATEDIFF('year', MIN(birthdate), current_date) as oldest_age, MIN(birthdate) as youngest_birthdate, DATEDIFF('year', MAX(birthdate), current_date) as youngest_age, FROM gold.dim_customers").df()
date_customer

,oldest_birthdate,oldest_age,youngest_birthdate,youngest_age
0,1916-02-10,109,1916-02-10,39


### 4. Measures Exploration

Measures are numeric fields like sales, quantity, or profit. This step looks at how these numbers are distributed and helps detect outliers, trends, and data quality issues.

##### Find the Total Sales

In [12]:
sales = con.execute("SELECT SUM(sales_amount) as total_sales from gold.fact_sales").df()
sales

,total_sales
0,29351258.0


##### Find how many items are sold

In [13]:
sold = con.execute("SELECT SUM(quantity) as total_quantity from gold.fact_sales").df()
sold

,total_quantity
0,60404.0


##### Find the Total number of Orders

In [14]:
orders = con.execute("SELECT COUNT(order_number) as total_orders FROM gold.fact_sales").df()
orders

,total_orders
0,60379


In [15]:
order_distinct = con.execute("SELECT COUNT(DISTINCT order_number) as total_orders FROM gold.fact_sales").df()
order_distinct

,total_orders
0,27657


##### Find the Total number of Products

In [16]:
n_products = con.execute("SELECT COUNT(product_name) as total_orders FROM gold.dim_products").df()
n_products

,total_orders
0,295


In [17]:
n_products_distinct = con.execute("SELECT COUNT(DISTINCT product_name) as total_orders FROM gold.dim_products").df()
n_products_distinct

,total_orders
0,295


##### Find the Total number of Customers

In [18]:
n_customers = con.execute("SELECT COUNT(customer_key) as total_customers FROM gold.dim_customers").df()
n_customers

,total_customers
0,18484


##### Find the Total number of Customers that has place an order

In [19]:
n_customer_sales = con.execute("SELECT COUNT(customer_key) as total_customers FROM gold.fact_sales").df()
n_customer_sales

,total_customers
0,60379


In [20]:
n_customer_sales_distinct = con.execute("SELECT COUNT(DISTINCT customer_key) as total_customers FROM gold.fact_sales").df()
n_customer_sales_distinct

,total_customers
0,18482


##### Generate a Report that shows all key metrics of the business

In [21]:
report = con.execute("""
SELECT 'Total Sales' AS measure_name, PRINTF('%.2f', CAST(SUM(sales_amount) AS DOUBLE)) AS measure_value FROM gold.fact_sales
UNION ALL
SELECT 'Total Quantity', PRINTF('%.0f', CAST(SUM(quantity) AS DOUBLE)) FROM gold.fact_sales
UNION ALL
SELECT 'Average Price', PRINTF('%.2f', CAST(AVG(price) AS DOUBLE)) FROM gold.fact_sales
UNION ALL
SELECT 'Total no. Orders', PRINTF('%.0f', CAST(COUNT(DISTINCT order_number) AS DOUBLE)) FROM gold.fact_sales
UNION ALL
SELECT 'Total no. Products', PRINTF('%.0f', CAST(COUNT(product_name) AS DOUBLE)) FROM gold.dim_products
UNION ALL
SELECT 'Total no. Customers', PRINTF('%.0f', CAST(COUNT(customer_key) AS DOUBLE)) FROM gold.dim_customers
""").df()

report

,measure_name,measure_value
0,Total Sales,29351258.00
1,Total Quantity,60404
2,Average Price,486.11
3,Total no. Orders,27657
4,Total no. Products,295
5,Total no. Customers,18484


### 5. Magnitude Analysis

This analysis tells you how much of something exists. It shows the scale of values in your data, helping you understand volume and size.

##### Find total customers by countries

In [22]:
n_customer_by_country = con.execute("SELECT COUNT(customer_id), country FROM gold.dim_customers GROUP BY country").df()
n_customer_by_country

,count(customer_id),country
0,1571,Canada
1,337,n/a
2,1780,Germany
3,1810,France
4,1913,United Kingdom
5,3591,Australia
6,7482,United States


##### Find total customers by gender

In [23]:
n_customer_by_gender = con.execute("SELECT COUNT(customer_id) as total_customers, gender FROM gold.dim_customers GROUP BY gender ORDER BY total_customers DESC").df()
n_customer_by_gender

,total_customers,gender
0,9341,Male
1,9128,Female
2,15,n/a


##### Find total products by category

In [24]:
n_products_by_category = con.execute("SELECT COUNT(product_id) as total_products, category FROM gold.dim_products GROUP BY category ORDER BY total_products DESC").df()
n_products_by_category

,total_products,category
0,127,Components
1,97,Bikes
2,35,Clothing
3,29,Accessories
4,7,n/a


##### What is  the average costs in each category? 

In [25]:
avg_cost_country = con.execute("SELECT category, AVG(cost) as avg_costs FROM gold.dim_products GROUP BY category ORDER BY avg_costs DESC").df()
avg_cost_country

,category,avg_costs
0,Bikes,949.443299
1,Components,264.716535
2,n/a,28.571429
3,Clothing,24.800000
4,Accessories,13.172414


##### What is the total revenue generated for each category

In [26]:
total_revenue_by_category = con.execute("SELECT p.category, SUM(f.sales_amount) as total_revenue FROM gold.fact_sales f LEFT JOIN gold.dim_products p on f.product_key = p.product_key GROUP BY p.category ORDER BY total_revenue DESC").df()
total_revenue_by_category

,category,total_revenue
0,Bikes,28311657.0
1,Accessories,699909.0
2,Clothing,339692.0


##### Find total revenue is generated by each customer

In [27]:
total_revenue_customer = con.execute("SELECT c.customer_key, c.first_name, c.last_name, SUM(f.sales_amount) as total_revenue FROM gold.fact_sales f LEFT JOIN gold.dim_customers c on c.customer_key = f.customer_key GROUP BY c.customer_key, c.first_name, c.last_name ORDER BY total_revenue DESC").df()
total_revenue_customer

,customer_key,first_name,last_name,total_revenue
0,1133,Kaitlyn,Henderson,13294.0
1,1302,Nichole,Nara,13294.0
2,1309,Margaret,He,13268.0
3,1132,Randall,Dominguez,13265.0
4,1301,Adriana,Gonzalez,13242.0
...,...,...,...,...
18477,17301,Abigail,Bennett,2.0
18478,17969,Marcus,Morgan,2.0
18479,16994,Hunter,Miller,2.0
18480,17095,Melanie,Peterson,2.0


##### What is the distribution of sold items across countries ? 

In [28]:
distribution_items_countries = con.execute("SELECT c.country, SUM(f.quantity) as total_sold_items FROM gold.fact_sales f LEFT JOIN gold.dim_customers c ON f.customer_key = c.customer_key GROUP BY c.country ORDER BY total_sold_items DESC").df()
distribution_items_countries

,country,total_sold_items
0,United States,20474.0
1,Australia,13344.0
2,Canada,7629.0
3,United Kingdom,6908.0
4,Germany,5626.0
5,France,5552.0
6,n/a,871.0


### 6. Ranking Analysis

Ranking helps identify top or bottom performers based on a metric.

##### Which 5 products generate the highest revenue ? 

In [29]:
products_generate_rank = con.execute("SELECT * FROM (SELECT p.product_name, SUM(f.sales_amount) as total_revvenue, RANK() OVER (ORDER BY SUM(f.sales_amount) DESC) as rank_products FROM gold.fact_sales f LEFT JOIN gold.dim_products p ON f.product_key = p.product_key GROUP BY p.product_name) t WHERE rank_products <= 5").df()
products_generate_rank

,product_name,total_revvenue,rank_products
0,Mountain-200 Black- 46,1373454.0,1
1,Mountain-200 Black- 42,1363128.0,2
2,Mountain-200 Silver- 38,1339394.0,3
3,Mountain-200 Silver- 46,1298709.0,4
4,Mountain-200 Black- 38,1292559.0,5


##### What are the 5 worst performing products in terms of sales ? 

In [30]:
worst_perform = con.execute("SELECT c.customer_key, c.first_name, c.last_name, COUNT(DISTINCT order_number) as total_orders FROM gold.fact_sales f LEFT JOIN gold.dim_customers c ON f.customer_key = c.customer_key GROUP BY c.customer_key, c.first_name, c.last_name ORDER BY total_orders LIMIT 10").df()
worst_perform

,customer_key,first_name,last_name,total_orders
0,2400,Edward,Young,1
1,5284,Clayton,Raji,1
2,2468,Kristopher,Martinez,1
3,5786,Christian,Lee,1
4,6725,Sheila,Blanco,1
5,1797,Jason,Zhang,1
6,7899,Monica,Chandra,1
7,510,Roger,Harui,1
8,6615,Mason,Cook,1
9,8555,Emmanuel,Raman,1


## Data & Report Analysis

### 7. Change Over Time

This analysis focuses on how metrics evolve over a period. It highlights trends, spikes, or declines.

In [31]:
change_over_time = con.execute("""
SELECT
strftime(order_date, '%Y-%b') as order_date,
SUM(sales_amount) as total_sales,
COUNT(DISTINCT customer_key) as total_customers,
SUM(quantity) as total_quantity
FROM gold.fact_sales
WHERE order_date IS NOT NULL
GROUP BY strftime(order_date, '%Y-%b')
ORDER BY strftime(order_date, '%Y-%b')
""").df()
change_over_time

,order_date,total_sales,total_customers,total_quantity
0,2010-Dec,43419.0,14,14.0
1,2011-Apr,502042.0,157,157.0
2,2011-Aug,614516.0,193,193.0
3,2011-Dec,669395.0,222,222.0
4,2011-Feb,466307.0,144,144.0
5,2011-Jan,469795.0,144,144.0
6,2011-Jul,596710.0,188,188.0
7,2011-Jun,737793.0,230,230.0
8,2011-Mar,485165.0,150,150.0
9,2011-May,561647.0,174,174.0


### 8. Cumulative Analysis

Here, you look at how values accumulate over time. It's great for understanding long-term trends and growth.

In [32]:
cumulative_analysis = con.execute("""
SELECT 
    order_date,
    total_sales,
    SUM(total_sales) OVER (ORDER BY order_date) AS running_total_sales,
    avg_price,
    avg(avg_price) OVER (ORDER BY order_date) AS moving_average
    FROM
        (
            SELECT
                DATETRUNC ('month', order_date) AS order_date,
                SUM(sales_amount) AS total_sales,
                AVG(price) as avg_price
                FROM gold. fact_sales
                WHERE order_date IS NOT NULL
                GROUP BY DATETRUNC ('month', order_date)
        ) t
""").df()
cumulative_analysis

,order_date,total_sales,running_total_sales,avg_price,moving_average
0,2010-12-01,43419.0,43419.0,3101.357143,3101.357143
1,2011-01-01,469795.0,513214.0,3262.465278,3181.911210
2,2011-02-01,466307.0,979521.0,3238.243056,3200.688492
3,2011-03-01,485165.0,1464686.0,3234.433333,3209.124702
4,2011-04-01,502042.0,1966728.0,3197.719745,3206.843711
5,2011-05-01,561647.0,2528375.0,3227.856322,3210.345813
6,2011-06-01,737793.0,3266168.0,3207.795652,3209.981504
7,2011-07-01,596710.0,3862878.0,3173.989362,3205.482486
8,2011-08-01,614516.0,4477394.0,3184.020725,3203.097846
9,2011-09-01,603047.0,5080441.0,3259.713514,3208.759413


### 9. Performance Analysis

Performance analysis evaluates how well something is doing compared to expectations or benchmarks.

In [33]:
performance_analysis = con.execute("""
WITH yearly_product_sales AS (
    SELECT 
    YEAR(f.order_date) as order_year,
    p.product_name,
    SUM(f.sales_amount) as current_sales
    FROM gold.fact_sales f 
    LEFT JOIN gold.dim_products p ON f.product_key = p.product_key 
    WHERE f.order_date IS NOT NULL
    GROUP BY YEAR(f.order_date), p.product_name
)

SELECT
    order_year, 
    product_name,
    current_sales,
    
    -- Average sales per product
    AVG(current_sales) OVER (PARTITION BY product_name) AS avg_sales,
    
    -- Difference from average
    current_sales - AVG(current_sales) OVER (PARTITION BY product_name) AS diff_avg,
    
    -- Above/Below/Average classification
    CASE
        WHEN current_sales > AVG(current_sales) OVER (PARTITION BY product_name) THEN 'Above Avg'
        WHEN current_sales < AVG(current_sales) OVER (PARTITION BY product_name) THEN 'Below Avg'
        ELSE 'Avg'
    END AS avg_change,
    
    -- Prior year sales
    LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) AS py_sales,
    
    -- Difference from prior year
    current_sales - LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) AS diff_py,
    
    -- Prior year change label
    CASE
        WHEN current_sales > LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) THEN 'Increase'
        WHEN current_sales < LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) THEN 'Decrease'
        ELSE 'No Change'
    END AS py_change

FROM yearly_product_sales
ORDER BY product_name, order_year;
""").df()
performance_analysis

,order_year,product_name,current_sales,avg_sales,diff_avg,avg_change,py_sales,diff_py,py_change
0,2012,AWC Logo Cap,72.0,6570.0,-6498.0,Below Avg,NaN,NaN,No Change
1,2013,AWC Logo Cap,18891.0,6570.0,12321.0,Above Avg,72.0,18819.0,Increase
2,2014,AWC Logo Cap,747.0,6570.0,-5823.0,Below Avg,18891.0,-18144.0,Decrease
3,2012,All-Purpose Bike Stand,159.0,13197.0,-13038.0,Below Avg,NaN,NaN,No Change
4,2013,All-Purpose Bike Stand,37683.0,13197.0,24486.0,Above Avg,159.0,37524.0,Increase
...,...,...,...,...,...,...,...,...,...
266,2014,Women's Mountain Shorts- L,1610.0,12705.0,-11095.0,Below Avg,23800.0,-22190.0,Decrease
267,2013,Women's Mountain Shorts- M,23380.0,12320.0,11060.0,Above Avg,NaN,NaN,No Change
268,2014,Women's Mountain Shorts- M,1260.0,12320.0,-11060.0,Below Avg,23380.0,-22120.0,Decrease
269,2013,Women's Mountain Shorts- S,20230.0,10640.0,9590.0,Above Avg,NaN,NaN,No Change


### 10. Part to Whole Propotional

This analysis shows how individual pieces contribute to the total. It’s often visualized using pie charts or percentage bars.

In [34]:
propotional = con.execute("""
    WITH category_sales as (SELECT category, SUM(sales_amount) total_sales FROM gold.fact_sales f LEFT JOIN gold.dim_products p ON f.product_key = p.product_key GROUP BY category)
                          
    SELECT 
        category,
        total_sales,
        SUM(total_sales) OVER () AS overall_sales,
        CONCAT(
            ROUND((CAST(total_sales AS FLOAT) / SUM(total_sales) OVER ()) * 100, 2),
            '%'
        ) AS percentage_of_total
    FROM category_sales;
""").df()
propotional

,category,total_sales,overall_sales,percentage_of_total
0,Bikes,28311657.0,29351258.0,96.46%
1,Clothing,339692.0,29351258.0,1.16%
2,Accessories,699909.0,29351258.0,2.38%


### 11. Data Segmentation

Segmentation means splitting your data into meaningful groups to analyze behavior or performance within each group.

In [35]:
data_segmentation = con.execute("""
    with customer_spending AS (
        SELECT
        c.customer_key,
        SUM(f.sales_amount) AS total_spending,
        MIN(order_date) AS first_order,
        MAX(order_date) AS last_order,
        DATEDIFF ('month', MIN(order_date), MAX(order_date)) AS lifespan
        FROM gold.fact_sales f
        LEFT JOIN gold.dim_customers c
        ON f.customer_key = c.customer_key
        GROUP BY c.customer_key
    )
                                
    SELECT
    customer_segment,
    COUNT (customer_key) as customer_key
    FROM (
        SELECT
            customer_key,
            CASE WHEN lifespan >= 12 AND total_spending > 5000 THEN 'VIP'
            WHEN lifespan >= 12 AND total_spending <= 5000 THEN 'Regular' ELSE 'New'
            END customer_segment
            FROM customer_spending
    ) t
    GROUP BY customer_segment
    ORDER BY customer_key DESC
""").df()
data_segmentation

,customer_segment,customer_key
0,New,14629
1,Regular,2200
2,VIP,1653


### 12. Reporting

Reporting brings together all analyses into readable summaries for decision-making. It combines visuals, KPIs, and insights to give a clear view of business performance.

Purpose: This report consolidates key customer metrics and behaviors


Highlights:
1. Gathers essential fields such as names, ages, and transaction details. 
2. Segments customers into categories (VIP, Regular, New) and age groups.
3. Aggregates customer-level metrics:
    - total orders
    - total sales
    - total quantity purchased
    - total products
    - lifespan (in months)
4. Calculates valuable KPIs:
    - recency (months since last order)
    - average order value
    - average monthly spend

In [36]:
reporting_customer = con.execute("""
CREATE VIEW IF NOT EXISTS gold.report_customer AS
WITH base_query AS (
    SELECT 
        f.order_number,
        f.product_key,
        f.order_date, 
        f.sales_amount, 
        f.quantity, 
        c.customer_key, 
        c.customer_number, 
        CONCAT(c.first_name,' ',c.last_name) as customer_name,
        DATEDIFF('year', c.birthdate, current_date) as age
        FROM gold.fact_sales f 
        LEFT JOIN gold.dim_customers c 
        ON f.customer_key = c.customer_key 
        WHERE order_date IS NOT NULL                        
),
customer_aggregation as (
    SELECT 
        customer_key, 
        customer_number, 
        customer_name,
        age,
        COUNT (DISTINCT order_number) as total_orders,
        SUM(sales_amount) as total_sales,
        SUM(quantity) as total_quantity,
        COUNT (DISTINCT product_key) as total_products,
        MAX(order_date) as last_order_date,
        DATEDIFF('month', MIN(order_date), MAX(order_date)) as lifespan
        FROM base_query
        GROUP BY customer_key, customer_number, customer_name, age
)           

SELECT
    customer_key,
    customer_number,
    customer_name,
    age,
    CASE
        WHEN age < 20 THEN 'Under 20'
        WHEN age between 20 and 29 THEN '20-29' 
        WHEN age between 30 and 39 THEN '30-39'
        WHEN age between 40 and 49 THEN '40-49' 
        ELSE '50 and above'
    END AS age_group,
    CASE
        WHEN lifespan >= 12 AND total_sales > 5000 THEN 'VIP'
        WHEN lifespan >= 12 AND total_sales <= 5000 THEN 'Regular' 
        ELSE 'New'
    END AS customer_segment,
    last_order_date,
    DATEDIFF ('month', last_order_date, current_date) as recency,
    total_orders,
    total_sales,
    total_quantity,
    total_products,
    lifespan,
    CASE 
        WHEN total_orders = 0 THEN 0 
        ELSE total_sales / total_orders 
    END AS avg_order_value,
    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_spend
    FROM customer_aggregation

          
""")

,Count


Product Report

Purpose:
This report consolidates key product metrics and behaviors.
Highlights:
1. Gathers essential fields such as product name, category, subcategory, and cost
2. Segments products by revenue to identify High-Performers, Mid-Range, or Low-Performers
3. Aggregates product-level metrics:
    - total orders
    - total sales
    - total quantity sold
    - total customers (unique)
    - lifespan (in months)
4. Calculates valuable KPIs:
    - recency (months since last sale) 
    - average order revenue (AOR) 
    - average monthly revenue


In [ ]:
reporting_products = con.execute("""

CREATE VIEW gold.report_products AS
WITH base_query AS (
    SELECT
        f.order_number,
        f.order_date,
        f.customer_key,
        f.sales_amount,
        f.quantity,
        p.product_key, 
        p.product_name,
        p.category,
        p.subcategory,                   
        p.cost
    FROM gold.fact_sales f
    LEFT JOIN gold.dim_products p
        ON f.product_key = p.product_key
    WHERE order_date IS NOT NULL                           
),
product_aggregations AS (
    SELECT
        product_key, 
        product_name, 
        category,
        subcategory,
        cost,
        DATEDIFF('month', MIN(order_date), MAX(order_date)) AS lifespan,
        MAX(order_date) AS last_sale_date,
        COUNT(DISTINCT order_number) AS total_orders,
        COUNT(DISTINCT customer_key) AS total_customers,
        SUM(sales_amount) AS total_sales,
        SUM(quantity) AS total_quantity,
        ROUND(
            AVG(CAST(sales_amount AS FLOAT) / NULLIF(quantity, 0)),
            1
        ) AS avg_selling_price
    FROM base_query
    GROUP BY
        product_key,
        product_name,
        category,
        subcategory,
        cost
)
SELECT
    product_key,
    product_name, 
    category,
    subcategory,
    cost,
    last_sale_date,
    DATEDIFF('month', last_sale_date, CURRENT_DATE) AS recency_in_months,
    CASE
        WHEN total_sales > 50000 THEN 'High-Performer'
        WHEN total_sales >= 10000 THEN 'Mid-Range'
        ELSE 'Low-Performer'
    END AS product_segment,
    lifespan,
    total_orders,
    total_sales,
    total_quantity,
    total_customers,
    avg_selling_price,
    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_revenue
FROM product_aggregations

""")

In [41]:
report_cus = con.execute("SELECT * FROM gold.report_customer").df()
report_cus

,customer_key,customer_number,customer_name,age,age_group,customer_segment,last_order_date,recency,total_orders,total_sales,total_quantity,total_products,lifespan,avg_order_value,avg_monthly_spend
0,2,AW00011001,Eugene Huang,49,40-49,VIP,2013-12-10,137,3,6384.0,11.0,10,35,2128.000000,182.400000
1,14,AW00011013,Ian Jenkins,46,40-49,New,2014-01-21,136,2,114.0,5.0,5,9,57.000000,12.666667
2,17,AW00011016,Wyatt Hill,41,40-49,New,2013-02-09,147,1,2332.0,3.0,3,0,2332.000000,2332.000000
3,40,AW00011039,Marc Martin,71,50 and above,VIP,2013-11-11,138,3,6636.0,9.0,8,33,2212.000000,201.090909
4,61,AW00011060,Jon Zhou,66,50 and above,VIP,2013-06-25,143,3,8153.0,6.0,6,27,2717.666667,301.962963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18477,18405,AW00029404,Chloe Ross,51,50 and above,New,2012-08-05,153,1,2049.0,1.0,1,0,2049.000000,2049.000000
18478,18406,AW00029405,Meagan Vance,42,40-49,New,2013-08-13,141,1,54.0,1.0,1,0,54.000000,54.000000
18479,18422,AW00029421,Blake Williams,44,40-49,New,2013-12-02,137,1,78.0,2.0,2,0,78.000000,78.000000
18480,18452,AW00029451,Geoffrey Lopez,66,50 and above,New,2013-02-08,147,1,25.0,2.0,2,0,25.000000,25.000000


In [43]:
report_prod = con.execute("SELECT * FROM gold.report_products").df()
report_prod

,product_key,product_name,category,subcategory,cost,last_sale_date,recency_in_months,product_segment,lifespan,total_orders,total_sales,total_quantity,total_customers,avg_selling_price,avg_monthly_revenue
0,287,HL Road Tire,Accessories,Tires and Tubes,12,2014-01-28,136,Mid-Range,13,858,28314.0,858.0,820,33.0,2178.000000
1,113,Mountain-500 Silver- 44,Bikes,Mountain Bikes,308,2013-12-25,137,Mid-Range,12,39,22035.0,39.0,39,565.0,1836.250000
2,277,Short-Sleeve Classic Jersey- S,Clothing,Jerseys,42,2014-01-28,136,Mid-Range,13,406,21924.0,406.0,402,54.0,1686.461538
3,294,Classic Vest- S,Clothing,Vests,24,2014-01-28,136,Mid-Range,13,168,10944.0,171.0,168,64.0,841.846154
4,272,Women's Mountain Shorts- L,Clothing,Shorts,26,2014-01-28,136,Mid-Range,12,363,25410.0,363.0,363,70.0,2117.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,Road-750 Black- 44,Bikes,Road Bikes,344,2013-12-28,137,High-Performer,12,360,194400.0,360.0,360,540.0,16200.000000
126,250,Long-Sleeve Logo Jersey- S,Clothing,Jerseys,38,2014-01-28,136,Mid-Range,12,429,21450.0,429.0,426,50.0,1787.500000
127,293,Classic Vest- M,Clothing,Vests,24,2014-01-27,136,Mid-Range,12,199,12736.0,199.0,199,64.0,1061.333333
128,119,Mountain-400-W Silver- 46,Bikes,Mountain Bikes,420,2013-12-24,137,High-Performer,11,138,106122.0,138.0,138,769.0,9647.454545
